In [ ]:
from google.colab import files
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
from textblob import TextBlob
from collections import Counter
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,LSTM
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints
import six
from tensorflow.keras.utils import deserialize_keras_object
from joblib import dump, load
from google.colab import files
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,Embedding,Flatten
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Dropout, GRU, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
import tensorflow as tf
 
from tensorflow.keras.layers import concatenate, Lambda


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# def preparing_data(file_name):
#     files=open(file_name,'r')
#     sentences=files.readlines()
#     files.close()
#     # print(lines)
#     sms_text=[]
#     english_text=[]

#     for index,sentence in enumerate(sentences):
#         if index%3==0:
#            sms_text.append(sentence.strip())
#         elif index%3==1:
#             english_text.append(sentence.strip())
#         else:
#             pass
#     return sms_text,english_text             



In [ ]:
# file_name='/content/drive/MyDrive/Applied Ai Course assignments/Assiginments/CS2/en2cn-2k.en2nen2cn'
# sms_text,english_text=preparing_data(file_name)
# print(len(sms_text), len(english_text))


In [ ]:
# data={'SMS_TEXT':sms_text,'ENGLISH_TEXT': english_text}
# data=pd.DataFrame(data)
# print(data.head(10))


In [ ]:
# data.head()

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Applied Ai Course assignments/sentence_correction/rawdata.csv')


In [ ]:
print("Data Shape",data.shape)      

Data Shape (2000, 2)


In [ ]:
data.head()

,SMS_TEXT,ENGLISH_TEXT
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [ ]:

def preprocessing_steps(data):
    """Applying the length on both sms_text and english_text and filtering the sentences based on length 
    adding start token and end token for inputs and output dataframe
    \t-> start token which represents start of the sentence
    \n-> end token which represents end of the sentence.
    Removing the sms_length, english_length, and ENGLISH_TEXT and appending ENGLISH_INPUT,ENGLISH_OUTPUT for the decoder."""
    data['sms_length']=data['SMS_TEXT'].str.split().apply(len)
    data['eng_length']=data['ENGLISH_TEXT'].str.split().apply(len)
    data=data[data['sms_length']<=39]
    data=data[data['eng_length']<=40]
    data['ENGLISH_INPUT']='<start> '+data['ENGLISH_TEXT'].astype(str)
    data['ENGLISH_OUTPUT']=data['ENGLISH_TEXT'].astype(str)+' <end>'
    data=data.drop(['sms_length','eng_length','ENGLISH_TEXT'],axis=1)
    return data


In [ ]:
preprocessed_data=preprocessing_steps(data)
print(preprocessed_data.shape)
preprocessed_data.iloc[0]['ENGLISH_INPUT']=str(preprocessed_data.iloc[0]['ENGLISH_INPUT'])+' <end>'
# preprocessed_data.iloc[0]['ENGLISH_OUTPUT']='\t ' + str(preprocessed_data.iloc[0]['ENGLISH_OUTPUT'])+' \n'

(1991, 3)


In [ ]:
preprocessed_data.head()

,SMS_TEXT,ENGLISH_INPUT,ENGLISH_OUTPUT
0,"U wan me to ""chop"" seat 4 u nt?",<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>
4,Hi! How did your week go? Haven heard from you...,<start> Hi! How did your week go? Haven't hear...,Hi! How did your week go? Haven't heard from y...


In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data= train_test_split(preprocessed_data,test_size=0.01, random_state=42)
print(train_data.shape)
print(test_data.shape)


(1971, 3)
(20, 3)


In [ ]:
train_data

,SMS_TEXT,ENGLISH_INPUT,ENGLISH_OUTPUT
396,Leaving ard that time too. Bringing laptop home?,<start> Leaving around that time too. Bringing...,Leaving around that time too. Bringing laptop ...
1974,Huh. If wont finish today that means i will pr...,"<start> Huh? If I won't finish today, that mea...","Huh? If I won't finish today, that means I wil..."
516,Ya la...No choice...Wat to do? U intro me some...,<start> Yes. No choice. What to do? You can in...,Yes. No choice. What to do? You can introduce ...
1671,"Hi, may i know ur handphone number","<start> Hi, may I know your handphone number?","Hi, may I know your handphone number? <end>"
351,Hey yun... Can you help me print something out...,<start> Hey Yun. Can you help me print someth...,Hey Yun. Can you help me print something out ...
...,...,...,...
1133,Haha.... Ya i don mind... U are not right?,<start> Haha. I don't mind. You are not right?,Haha. I don't mind. You are not right? <end>
1298,Blk 295 punggol central.,<start> Block 295 Punggol Central.,Block 295 Punggol Central. <end>
862,Its ok. Hi ranger u m or f? Workg? Age? Its ok...,"<start> It's ok. Hi, ranger you male or female...","It's ok. Hi, ranger you male or female? Workin..."
1463,Hey yun ask you ah... where did you the answer...,"<start> Hey Yun, can I ask you? Where did you ...","Hey Yun, can I ask you? Where did you get the ..."


In [ ]:

tokenizer = Tokenizer(filters=" ",char_level=False,lower=False)
print("SMS_TEXT")
tokenizer.fit_on_texts(train_data['SMS_TEXT'].values)
print("English text")
tokenizer_e = Tokenizer(filters=" ",char_level=False,lower=False)
tokenizer_e.fit_on_texts(train_data['ENGLISH_INPUT'].values)


SMS_TEXT
English text


In [ ]:
encoder_vocab_size=len(tokenizer.word_index.keys())
print(encoder_vocab_size)
decoder_vocab_size=len(tokenizer_e.word_index.keys())
print(decoder_vocab_size)


6943
5266


In [ ]:
tokenizer.word_index

{'u': 1,
 'to': 2,
 'i': 3,
 'I': 4,
 'at': 5,
 'go': 6,
 'a': 7,
 'me': 8,
 'ü': 9,
 'my': 10,
 'the': 11,
 'e': 12,
 'for': 13,
 'in': 14,
 'U': 15,
 'got': 16,
 'can': 17,
 'you': 18,
 'is': 19,
 'on': 20,
 'ur': 21,
 '2': 22,
 'not': 23,
 'of': 24,
 'n': 25,
 'so': 26,
 '4': 27,
 'Hey': 28,
 'So': 29,
 'or': 30,
 'time': 31,
 'will': 32,
 'wan': 33,
 'lor...': 34,
 'r': 35,
 "I'm": 36,
 'we': 37,
 'it': 38,
 'meet': 39,
 'have': 40,
 'be': 41,
 'dun': 42,
 'all': 43,
 'no': 44,
 'Ü': 45,
 'and': 46,
 'if': 47,
 'still': 48,
 'then': 49,
 'one': 50,
 'do': 51,
 'get': 52,
 'are': 53,
 'going': 54,
 'like': 55,
 'out': 56,
 'come': 57,
 'but': 58,
 'up': 59,
 'But': 60,
 'your': 61,
 'Hi': 62,
 'b': 63,
 'ask': 64,
 'how': 65,
 'wat': 66,
 'goin': 67,
 "i'm": 68,
 'den': 69,
 'when': 70,
 'think': 71,
 'call': 72,
 'w': 73,
 'Haha...': 74,
 'with': 75,
 'home': 76,
 'now': 77,
 'free': 78,
 'back': 79,
 'that': 80,
 'see': 81,
 'buy': 82,
 'la...': 83,
 'she': 84,
 'How': 85,
 'juz':

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [ ]:
# class Dataset:
#     def __init__(self, data, tknizer_ita, tknizer_eng, max_len):
#         self.encoder_inps = data['italian'].values
#         self.decoder_inps = data['d_english_input'].values
#         self.decoder_outs = data['d_english_output'].values
#         self.tknizer_eng = tknizer_eng
#         self.tknizer_ita = tknizer_ita
#         self.max_len = max_len

#     def __getitem__(self, i):
#         self.encoder_seq = self.tknizer_ita.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
#         self.decoder_inp_seq = self.tknizer_eng.texts_to_sequences([self.decoder_inps[i]])
#         self.decoder_out_seq = self.tknizer_eng.texts_to_sequences([self.decoder_outs[i]])

#         self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
#         self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
#         self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
#         return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

#     def __len__(self): 
#         return len(self.encoder_inps)

    
# class Dataloder(tf.keras.utils.Sequence):    
#     def __init__(self, dataset, batch_size=1):
#         self.dataset = dataset
#         self.batch_size = batch_size
#         self.indexes = np.arange(len(self.dataset.encoder_inps))


#     def __getitem__(self, i):
#         start = i * self.batch_size
#         stop = (i + 1) * self.batch_size
#         data = []
#         for j in range(start, stop):
#             data.append(self.dataset[j])

#         batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
#         # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
#         return tuple([[batch[0],batch[1]],batch[2]])

#     def __len__(self):  # your model.fit_gen requires this function
#         return len(self.indexes) // self.batch_size

#     def on_epoch_end(self):
#         self.indexes = np.random.permutation(self.indexes)

In [ ]:
# class Dataset:
#     def __init__(self, data, tknizer_ita, tknizer_eng, max_len):
#         self.encoder_inps = data['SMS_TEXT'].values
#         self.decoder_inps = data['ENGLISH_INPUT'].values
#         self.decoder_outs = data['ENGLISH_OUTPUT'].values
#         self.tknizer_eng = tknizer_eng
#         self.tknizer_ita = tknizer_ita
#         self.max_len = max_len
#         # self.max_len1=max_len1
 
#     def __getitem__(self, i):
#         self.encoder_seq = self.tknizer_ita.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
#         self.decoder_inp_seq = self.tknizer_eng.texts_to_sequences([self.decoder_inps[i]])
#         self.decoder_out_seq = self.tknizer_eng.texts_to_sequences([self.decoder_outs[i]])
 
#         self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
#         self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
#         self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
#         return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq
 
#     def __len__(self): # your model.fit_gen requires this function
#         return len(self.encoder_inps)
 
    
# class Dataloder(tf.keras.utils.Sequence):    
#     def __init__(self, dataset, batch_size=1):
#         self.dataset = dataset
#         self.batch_size = batch_size
#         self.indexes = np.arange(len(self.dataset.encoder_inps))
 
 
#     def __getitem__(self, i):
#         start = i * self.batch_size
#         stop = (i + 1) * self.batch_size
#         data = []
#         for j in range(start, stop):
#             data.append(self.dataset[j])
 
#         batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
#         # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
#         return tuple([[batch[0],batch[1]],batch[2]])
 
#     def __len__(self):  # your model.fit_gen requires this function
#         return len(self.indexes) // self.batch_size
 
#     def on_epoch_end(self):
#         self.indexes = np.random.permutation(self.indexes)

# Attention

In [ ]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        
      super().__init__()

      self.e_embed = Embedding(input_dim=inp_vocab_size, output_dim=embedding_size, input_length= input_length)
      self.lstm_size = lstm_size
      
      self.e_lstm = LSTM(lstm_size, return_sequences=True, return_state=True)


    def call(self,input_sequence,states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        embedding = self.e_embed(input_sequence)
        # print(embedding.shape)
        # output_state, enc_h, enc_c = self.e_lstm( embedding, initial_state = states)
        output_state, enc_h, enc_c = self.e_lstm( embedding)

        return output_state, enc_h, enc_c

      

    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return [tf.zeros((batch_size, self.lstm_size)), tf.zeros((batch_size, self.lstm_size))]



<font color='blue'>**Attention**</font>

In [ ]:
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,scoring_function, att_units):

    super(Attention, self).__init__()

    self.scoring_function = scoring_function


    # Please go through the reference notebook and research paper to complete the scoring functions

    if self.scoring_function=='dot':
      # Intialize variables needed for Dot score function here
      self.dot= tf.keras.layers.Dot(axes = (1,2))      
      pass
    elif scoring_function == 'general':
      # Intialize variables needed for General score function here
      self.w= Dense(att_units)
      self.dot = tf.keras.layers.Dot(axes = (1,2))
      pass
    elif scoring_function == 'concat':
      # Intialize variables needed for Concat score function here
      self.w1= Dense(att_units)
      self.w2= Dense(att_units)
      self.v= Dense(1)
      pass
  
  
  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    decoder_hidden_state = tf.expand_dims(decoder_hidden_state, 1)
    if self.scoring_function == 'dot':
        # Implement Dot score function here
        score = tf.transpose(self.dot([tf.transpose(decoder_hidden_state, (0,2,1)), encoder_output]), (0,2,1))
        # print("In Dot")

        pass
    elif self.scoring_function == 'general':
        # Implement General score function here
        # print(encoder_output.shape)
        mulpy = self.w(encoder_output)
        # print(mulpy.shape)
        score = tf.transpose(self.dot([tf.transpose(decoder_hidden_state, (0, 2, 1)), mulpy]), (0, 2,1),)

        pass
    elif self.scoring_function == 'concat':
        inte = self.w1(decoder_hidden_state) + self.w2(encoder_output)
        tan = tf.nn.tanh(inte)
        score = self.v(tan)

    att_weights = tf.nn.softmax(score, axis =1)
    context_vector = att_weights * encoder_output
    context_vector = tf.reduce_sum( context_vector, axis=1)
    return context_vector, att_weights

        
    
    

<font color='blue'>**OneStepDecoder**</font>

In [ ]:
class OneStepDecoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):

      # Initialize decoder embedding layer, LSTM and any other objects needed
      super(OneStepDecoder, self).__init__()
      self.dec_embed = Embedding(input_dim = tar_vocab_size, output_dim = embedding_dim)
      self.lstm = LSTM(dec_units, return_sequences = True, return_state = True)
      self.attention = Attention(scoring_function = score_fun, att_units = att_units)
      self.den = Dense(tar_vocab_size)


  def call(self,input_to_decoder, encoder_output, state_h,state_c):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    embd = self.dec_embed(input_to_decoder)
    context_vec, attention_weights = self.attention( state_h,  encoder_output)    

    f_inp = tf.concat([tf.expand_dims(context_vec, 1), embd], axis = -1)
    # print(f_inp.shape)
    out, dec_h, dec_c = self.lstm(f_inp, [state_h, state_c])
    out = tf.reshape( out, (-1, out.shape[2]))
    output = self.den(out)

    return output, dec_h, dec_c, attention_weights, context_vec


In [ ]:
#  model2.layers[1].onestepdecoder(dec_input,encoder_outputs,
#                                                                                state_h,state_c,training=False)

<font color='blue'>**Decoder**</font>

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super(Decoder, self).__init__()
      self.input_length = input_length
      self.out_vocab_size = out_vocab_size
      self.oneStepDecoder = OneStepDecoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)
      self.out_vocab_size = out_vocab_size
        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook
        
        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
        # outputs = tf.TensorArray(dtype =  tf.float32, size= input_to_decoder.shape[1])
        outputs = tf.TensorArray(dtype =  tf.float32, size= tf.shape(input_to_decoder)[1])

        
        for timestep in range(tf.shape(input_to_decoder)[1]):

            output, decoder_hidden_state, decoder_cell_state, _, _ = self.oneStepDecoder(input_to_decoder[:, timestep:timestep+1],
                                                                                          encoder_output,decoder_hidden_state,decoder_cell_state)                                                                                            
                                                                                             
                                                                                             
            # Store the output in tensorarray
            outputs = outputs.write(timestep, output)
        # Return the tensor array
        outputs = tf.transpose(outputs.stack(), (1, 0, 2))
        return outputs
        
        
    

<font color='blue'>**Encoder Decoder model**</font>

In [ ]:
class encoder_decoder(tf.keras.Model):
  def __init__(self,inp_vocab_size, out_vocab_size,input_length, enc_units,embedding_dim, dec_units, max_len, score_fun, att_units, batch_size):
    #Intialize objects from encoder decoder
    super(encoder_decoder, self).__init__()
    self.encoder = Encoder(inp_vocab_size= inp_vocab_size +1, embedding_size=embedding_dim, lstm_size= att_units,input_length= max_len)
    self.decoder = Decoder(out_vocab_size +1, embedding_dim, input_length, dec_units ,score_fun ,att_units)
    self.batch_size = batch_size

  
  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    e_inp, d_inp = data[0], data[1]
    # print(data[0].shape)
    # print(data[1].shape)


    initial_state = self.encoder.initialize_states(self.batch_size)

    e_output, enc_h, enc_c = self.encoder(e_inp,initial_state)
    outputs = tf.TensorArray(dtype = tf.float32, size= 20)
        
    dec_h = enc_h
    dec_c = enc_c
    # print(dec_h.shape)
    output=self.decoder(d_inp,e_output, dec_h, dec_c)
  
    return output




<font color='blue'>**Custom loss function**</font>

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')

def custom_lossfunction(targets,logits):
  mask = tf.math.logical_not(tf.math.equal(targets, 0))
  loss_ = loss_object(targets, logits)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

# def custom_lossfunction(targets,logits):

  # Custom loss function that will not consider the loss for padded zeros.
  # Refer https://www.tensorflow.org/tutorials/text/nmt_with_attention#define_the_optimizer_and_the_loss_function


<font color='blue'>**Training**</font>

Implement dot function here.

In [ ]:
# Implement teacher forcing while training your model. You can do it two ways.
# Prepare your data, encoder_input,decoder_input and decoder_output
# if decoder input is 
# <start> Hi how are you
# decoder output should be
# Hi How are you <end>
# i.e when you have send <start>-- decoder predicted Hi, 'Hi' decoder predicted 'How' .. e.t.c

# or
 
# model.fit([train_ita,train_eng],train_eng[:,1:]..)
# Note: If you follow this approach some grader functions might return false and this is fine.

In [ ]:
class Dataset:
    def __init__(self, data, tknizer_ita, tknizer_eng, max_len,max_len1):
        self.encoder_inps = data['SMS_TEXT'].values
        self.decoder_inps = data['ENGLISH_INPUT'].values
        self.decoder_outs = data['ENGLISH_OUTPUT'].values
        self.tknizer_eng = tknizer_eng
        self.tknizer_ita = tknizer_ita
        self.max_len = max_len
        self.max_len1=max_len1
 
    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_ita.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_eng.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_eng.texts_to_sequences([self.decoder_outs[i]])
 
        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len1, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len1, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq
 
    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)
 
    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))
 
 
    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
 
        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])
 
    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size
 
    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [ ]:
train_dataset = Dataset(train_data, tokenizer, tokenizer_e, 39,40)
test_dataset  = Dataset(test_data, tokenizer, tokenizer_e, 39,40)
 
train_dataloader = Dataloder(train_dataset, batch_size=64)
test_dataloader = Dataloder(test_dataset, batch_size=20)


In [ ]:
# model = encoder_decoder(len(encoder_vocabluary), len(decoder_vocabluary),input_length, lstm_size, embedding_dim, att_units, max_len, 'dot', att_units, batch_size)
# model.compile(optimizer = 'Adam', loss = custom_lossfunction)



In [ ]:
from tensorflow.keras import backend as K
def accuracy(y_true, y_pred):

    pred_value= K.cast(K.argmax(y_pred, axis=-1), dtype='float32')
    true_value = K.cast(K.equal(y_true, pred_value), dtype='float32')

    mask = K.cast(K.greater(y_true, 0), dtype='float32')
    n_correct = K.sum(mask * true_value)
    n_total = K.sum(mask)
  
    return n_correct / n_total

In [ ]:
lstm_size = 100
embedding_dim =128
att_units = 100
dec_units=100
batch_size = 64 

In [ ]:
tf.keras.backend.clear_session()
model2  = encoder_decoder(encoder_vocab_size,decoder_vocab_size,39,lstm_size,embedding_dim,att_units, 40,'concat',dec_units,batch_size)
optimizer = tf.keras.optimizers.Adam(0.01)
model2.compile(optimizer=optimizer,loss=custom_lossfunction,metrics=[accuracy])
train_steps=train_data.shape[0]//64
valid_steps=test_data.shape[0]//20
model2.fit(train_dataloader, steps_per_epoch=train_steps, epochs=50, validation_data=test_dataloader, validation_steps=valid_steps)


Epoch 1/50
30/30 [==============================] - 30s 768ms/step - loss: 2.7525 - accuracy: 0.0712 - val_loss: 2.0542 - val_accuracy: 0.0898
Epoch 2/50
30/30 [==============================] - 22s 726ms/step - loss: 2.5893 - accuracy: 0.0767 - val_loss: 2.0270 - val_accuracy: 0.0938
Epoch 3/50
30/30 [==============================] - 22s 716ms/step - loss: 2.5480 - accuracy: 0.0816 - val_loss: 1.9935 - val_accuracy: 0.0977
Epoch 4/50
30/30 [==============================] - 21s 709ms/step - loss: 2.5218 - accuracy: 0.0853 - val_loss: 1.9663 - val_accuracy: 0.1055
Epoch 5/50
30/30 [==============================] - 22s 716ms/step - loss: 2.4275 - accuracy: 0.0935 - val_loss: 1.9318 - val_accuracy: 0.1133
Epoch 6/50
30/30 [==============================] - 22s 719ms/step - loss: 2.2555 - accuracy: 0.1220 - val_loss: 1.8514 - val_accuracy: 0.1172
Epoch 7/50
30/30 [==============================] - 22s 720ms/step - loss: 2.0829 - accuracy: 0.1552 - val_loss: 1.7958 - val_accuracy: 0.1328

In [ ]:
# rlp = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss', factor=0.1, patience=5, verbose=10,
#     mode='auto', min_delta=0.00001, cooldown=0, min_lr=0.0001,
# )

## <font color='blue'>**Inference**</font>

<font color='blue'>**Plot attention weights**</font>

In [ ]:
# https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/

from math import log
from numpy import array
from numpy import argmax
 
# beam search
def beam_search_decoder(data, k):
  sequences = [[list(), 0.0]]
  # print("asd")
  for row in data:
    all_candidates = list()
    for i in range(len(sequences)):
      seq, score = sequences[i]
      for j in range(len(row)):
        try:
          candidate = [seq + [j], score - log(row[j])]
          all_candidates.append(candidate)
        except ValueError as e:
          candidate = [seq + [j], 0]
          all_candidates.append(candidate)
      # order all candidates by score
    ordered = sorted(all_candidates, key=lambda tup:tup[1])
    # select k best
    sequences = ordered[:k]
  return sequences

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

def evaluate(sentence):
  max_length_targ = 40
  max_length_inp  = 39
  
  attention_plot = np.zeros((max_length_targ, max_length_inp))
  # sentence = preprocess(sentence)
  # sentence = sentence.strip()
  # print(sentence)
  #pdb.set_trace()
  inputs = [] 
  # for word in sentence.split():
  #     print(word)
  #     inputs.append(tokenizer.word_index[word])
  inputs = tokenizer.texts_to_sequences([sentence])
  inputs =  tf.keras.preprocessing.sequence.pad_sequences(inputs,maxlen=max_length_inp,padding='post') 
  inputs = tf.convert_to_tensor(inputs)
  result = '' 

  initial_state=model2.layers[0].initialize_states(batch_size=1)
  encoder_outputs, state_h,state_c = model2.layers[0](inputs,initial_state)   
  dec_input = tf.expand_dims([tokenizer_e.word_index['<start>']], 0)
  # dec_input=tf.expand_dims([tokenizer_e.word_index['\t']],0)


  for i in range(max_length_targ):
    Output,state_h,state_c,att_weights,_ = model2.layers[1].oneStepDecoder(dec_input,encoder_outputs,state_h,state_c)
    #Beam Search Decoder
    Result_beam_list=beam_search_decoder(Output,k=1)
    Result_beam=Result_beam_list[0][0]

    attention_weights = tf.reshape(att_weights, (-1, ))
    attention_plot[i]  = attention_weights.numpy()

    predicted_id = tf.argmax(Output[0]).numpy()
  
    result += tokenizer_e.index_word[Result_beam[0]] + ' '
    if tokenizer_e.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
  return result, sentence, attention_plot

In [ ]:
tokenizer_e.word_index["<end>"]

3899

In [ ]:
import matplotlib.pyplot as plt
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    fontdict = {'fontsize': 14}
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)  
    ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(mticker.MultipleLocator(1))
    plt.show()

In [ ]:
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    print("-"*50)
    # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    # plot_attention(attention_plot, sentence.split(' '),    result.split(' '))
    return result

In [ ]:
result=translate(test_data["SMS_TEXT"][889])


Input: fun..How i wish i can spend this coming v day with u..Hee..Can?Future guy
Predicted translation: I am 25 male. Chinese Malaysian. <end> 
--------------------------------------------------


In [ ]:
result=translate(test_data["SMS_TEXT"][488])


Input: I will come at 8.40pm
Predicted translation: I will be working until also will be ok with me and CC my place and that will you be 2:45. Then I will wait for me also. <end> 
--------------------------------------------------


In [ ]:
tokenizer.word_index["you."]

685

In [ ]:
test_data["SMS_TEXT"]

889     fun..How i wish i can spend this coming v day ...
1675       Are you going to send a mail... Tmr i cant leh
416     i cant understand my linear algebra tutor. i t...
1083    Yupz... Muz stick e foto on e booklet ma... Ok...
1105                Lazing around at home lor... u lei...
1000                              Can just go down right?
1219    Hey ü wanna meet outside e lt? Haha... Time's ...
1440    ANGEL y no reply leh?if u wan u can msg me at ...
769     Me n my sis eatin bfast... Hee... U call me lo...
1809    Exercise til wat time? Aft exercise la...Fat fat.
452         Guess wat im e one whose gonna b late..gee...
488                                 I will come at 8.40pm
1986    My sis so bo liao u noe. She arguing w me abt ...
1308          545 can... Cos i finish work at tis time...
1829    I already outside pl come early. reach here, c...
1184    ya.. found him already la.. of cos.. wat time ...
124     Just came to nydc n she just ordered a baked r...
819     Nope..

In [ ]:
import nltk.translate.bleu_score as bleu
from tqdm import tqdm
def BleuScore(validation):
    input=list(validation['SMS_TEXT'])
    Y_true=list(validation['ENGLISH_OUTPUT'])
    results = []
    bleuscores=[]
    input_sent_list = []
    # print(len(input))
    for i in tqdm(range(len(input))):
      try:
        # print("1")
        result, sentence, attention_plot = evaluate(input[i])
        # print(result)
        
        results.append(result)  
        input_sent_list.append(sentence)    
        bleuscores.append(bleu.sentence_bleu(Y_true[i], result))
        # print(":sss")
      except KeyError as e:
        pass  
    return sum(bleuscores)/len(bleuscores),bleuscores,results,input_sent_list

AvearageScore,bleuscores,results,input_sent_list=BleuScore(test_data)

print("Average Bleuscore:",AvearageScore)

100%|██████████| 20/20 [14:27<00:00, 43.37s/it]

Average Bleuscore: 0.7247005670864131


<font color='blue'>**Predict the sentence translation**</font>

In [ ]:

def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <\t> as input to decoder. and encoder final states as input_states to onestepdecoder.
  D. till we reach max_length of decoder or till the model predicted character <\n>:
         predictions, input_states, attention_weights = model.layers[1].onestepdecoder(input_to_decoder, encoder_output, input_states)
         And get the character using the tokenizer(character index) and then store it in a string.
  E. Return the predicted sentence

  '''


  input_sequence=tokenizer.texts_to_sequences([input_sentence])
  

  inputs=pad_sequences(input_sequence,maxlen=39,padding='post')
  inputs=tf.convert_to_tensor(inputs)
  result=''
  units=100
  hidden=[tf.zeros((1,units))]
  encoder_output,hidden_state,cell_state=model2.layers[0](inputs,hidden)
  dec_hidden=hidden_state
  dec_input=tf.expand_dims([tokenizer_e.word_index['<start>']],0)
  for t in range(40):
      predictions,dec_hidden,cell_state,attention_weights,context_vector=model2.layers[1].oneStepDecoder(dec_input,encoder_output,dec_hidden,cell_state)

      predicted_id=tf.argmax(predictions[0]).numpy()
      result+=tokenizer_e.index_word[predicted_id] + ' '
      if tokenizer_e.word_index['<end>']==predicted_id:
          return result
      dec_input= tf.expand_dims([predicted_id],0)
  return result

In [ ]:
for index in range(len(test_data)):
    print('*'*200)
    print("SMS_TEXT")
    print(test_data['SMS_TEXT'].values[index])
    print("Prediction")
    print(predict(test_data['SMS_TEXT'].values[index]))
    print("Original Sentence")
    print(test_data['ENGLISH_OUTPUT'].iloc[index])


********************************************************************************************************************************************************************************************************
SMS_TEXT
fun..How i wish i can spend this coming v day with u..Hee..Can?Future guy
Prediction
I am 25 male. Chinese Malaysian. <end> 
Original Sentence
Fun. How I wish I can spend this coming valentine's day with you. Hee. Can? Future guy. <end>
********************************************************************************************************************************************************************************************************
SMS_TEXT
Are you going to send a mail... Tmr i cant leh
Prediction
Are you free to the canteen for dessert. Do you want to meet up. <end> 
Original Sentence
Are you going to send a mail? Tomorrow I can't. <end>
*********************************************************************************************************************************************

In [ ]:
import nltk.translate.bleu_score as bleu
sum=0
for index in range(len(test_data)):
    reference=test_data['ENGLISH_OUTPUT'].iloc[index]
    translation=predict(test_data['SMS_TEXT'].iloc[index])
    sum+=bleu.sentence_bleu(reference,translation)
print(sum/len(test_data))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.6780770304835106
